### Projeto comparação entre algoritmos e modelos de Computação evolutiva

Achando o diretório em que esse arquivo se encontra 

In [72]:
import sys
import os

# Obtém o caminho do diretório do script atual
script_dir = os.getcwd()

# Adiciona o diretório pai ao caminho do sistema
sys.path.append(script_dir)

In [73]:
import pygame # Nos repositórios de referência essa lib é sempre importada tendo em vista que o gymnasium (gym) utiliza a mesma, deve evitar problemas

import time # será utilizada para coletar o período temporal necessário para que algumas funções sejam executadas
import numpy as np # numpy por motivos óbvios  

import torch # Todas as redes neurais serão criados utilizando PyTorch
import torch.nn as nn
import torch.optim as optim

from redes_neurais.rede_MLP import MLP
from func_aux.auxiliares import *

import matplotlib.pyplot as plt # Demonstração de Gráficos e imagens 

In [74]:
# usar GPU compatível com CUDA costuma ser mais rápido
if torch.cuda.is_available():
    is_gpu = torch.device("cuda")
else:
    is_gpu = torch.device("cpu")

torch.set_default_device(is_gpu)

In [75]:
import gymnasium as gym

In [76]:
def joga_jogo(env, rede_neural):

    if isinstance(env, str):
        env = gym.make(env)

    imagem, _ = env.reset()
    passos = 0
    game_recompenca = 0

    while True: # inicializa o loop do jogo
        passos += 1 # conta quantas vezes a IA foi requisitada ate o complecionismo do mesmo

        acao = escolhe_acao(rede_neural, imagem)

        nova_imagem, recompensa, finalizado, truncado, _ = env.step(acao)
        done = finalizado or truncado
        game_recompenca += recompensa

        imagem = nova_imagem

        if done:
            return rede_neural, passos, game_recompenca

In [133]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []

    recompencas_ao_tempo = []
    Tempo_por_geracao = []

    contador_recompencas = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        inicio = time.time()
        recompencas = []

        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
    
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        recompencas_ao_tempo.append(recompencas)
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        fim = time.time()
        Tempo_por_geracao.append(fim - inicio)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
    
    obs = [recompencas_ao_tempo, Tempo_por_geracao]
    return selecao_final[0], obs

In [134]:
env = gym.make("CartPole-v1")

redes_analisadas_1 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = True"
    }

    redes_analisadas_1.append([
        rede_melhor, informacoes_Acrobot
    ])


Geração: 1/20, Recompensa: [28.0, 19.0, 18.0]
Geração: 2/20, Recompensa: [110.0, 61.0, 57.0]
Geração: 3/20, Recompensa: [51.0, 46.0, 44.0]
Geração: 4/20, Recompensa: [71.0, 58.0, 49.0]
Geração: 5/20, Recompensa: [123.0, 76.0, 47.0]
Geração: 6/20, Recompensa: [189.0, 147.0, 71.0]
Geração: 7/20, Recompensa: [322.0, 179.0, 83.0]
Geração: 8/20, Recompensa: [500.0, 217.0, 202.0]
Geração: 9/20, Recompensa: [500.0, 319.0, 300.0]
Geração: 1/20, Recompensa: [48.0, 11.0, 10.0]
Geração: 2/20, Recompensa: [103.0, 66.0, 65.0]
Geração: 3/20, Recompensa: [500.0, 500.0, 110.0]
Geração: 1/20, Recompensa: [59.0, 42.0, 21.0]
Geração: 2/20, Recompensa: [72.0, 57.0, 53.0]
Geração: 3/20, Recompensa: [95.0, 54.0, 46.0]
Geração: 4/20, Recompensa: [96.0, 90.0, 68.0]
Geração: 5/20, Recompensa: [185.0, 154.0, 130.0]
Geração: 6/20, Recompensa: [194.0, 179.0, 177.0]
Geração: 7/20, Recompensa: [316.0, 243.0, 147.0]
Geração: 8/20, Recompensa: [500.0, 281.0, 226.0]
Geração: 9/20, Recompensa: [500.0, 500.0, 500.0]
Ger

In [135]:
env = gym.make("CartPole-v1")

redes_analisadas_2 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = False,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = False"
    }

    redes_analisadas_2.append([
        rede_melhor, informacoes_Acrobot
    ])


Geração: 1/20, Recompensa: [67.0, 17.0, 11.0]
Geração: 2/20, Recompensa: [81.0, 80.0, 51.0]
Geração: 3/20, Recompensa: [145.0, 62.0, 53.0]
Geração: 4/20, Recompensa: [170.0, 55.0, 43.0]
Geração: 5/20, Recompensa: [500.0, 244.0, 102.0]
Geração: 6/20, Recompensa: [466.0, 101.0, 96.0]
Geração: 7/20, Recompensa: [356.0, 203.0, 97.0]
Geração: 8/20, Recompensa: [278.0, 165.0, 134.0]
Geração: 9/20, Recompensa: [500.0, 500.0, 500.0]
Geração: 1/20, Recompensa: [68.0, 44.0, 25.0]
Geração: 2/20, Recompensa: [155.0, 54.0, 36.0]
Geração: 3/20, Recompensa: [83.0, 50.0, 46.0]
Geração: 4/20, Recompensa: [77.0, 63.0, 57.0]
Geração: 5/20, Recompensa: [386.0, 116.0, 95.0]
Geração: 6/20, Recompensa: [137.0, 78.0, 67.0]
Geração: 7/20, Recompensa: [112.0, 101.0, 85.0]
Geração: 8/20, Recompensa: [173.0, 139.0, 108.0]
Geração: 9/20, Recompensa: [87.0, 85.0, 80.0]
Geração: 10/20, Recompensa: [127.0, 80.0, 51.0]
Geração: 11/20, Recompensa: [101.0, 95.0, 84.0]
Geração: 12/20, Recompensa: [171.0, 74.0, 66.0]
Gera

In [80]:
env = gym.make("Acrobot-v1")

redes_analisadas_3 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = -65
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = True"
    }

    redes_analisadas_3.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [-84.0, -159.0, -232.0]
Geração: 2/20, Recompensa: [-92.0, -96.0, -156.0]
Geração: 3/20, Recompensa: [-92.0, -93.0, -132.0]
Geração: 4/20, Recompensa: [-92.0, -92.0, -96.0]
Geração: 5/20, Recompensa: [-82.0, -89.0, -94.0]
Geração: 6/20, Recompensa: [-83.0, -92.0, -93.0]
Geração: 7/20, Recompensa: [-84.0, -87.0, -100.0]
Geração: 8/20, Recompensa: [-86.0, -86.0, -91.0]
Geração: 9/20, Recompensa: [-71.0, -78.0, -82.0]
Geração: 10/20, Recompensa: [-72.0, -78.0, -78.0]
Geração: 11/20, Recompensa: [-63.0, -64.0, -69.0]
Geração: 12/20, Recompensa: [-63.0, -64.0, -74.0]
Geração: 13/20, Recompensa: [-63.0, -63.0, -64.0]
Geração: 14/20, Recompensa: [-65.0, -69.0, -71.0]
Geração: 15/20, Recompensa: [-64.0, -64.0, -73.0]
Geração: 16/20, Recompensa: [-70.0, -76.0, -81.0]
Geração: 17/20, Recompensa: [-71.0, -76.0, -77.0]
Geração: 18/20, Recompensa: [-70.0, -80.0, -84.0]
Geração: 19/20, Recompensa: [-76.0, -83.0, -84.0]
Geração: 20/20, Recompensa: [-69.0, -71.0, -72.0]
Gera

In [87]:
env = gym.make("Acrobot-v1")

redes_analisadas_4 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = False,
        learning_rate = 0.1,
        objetivo = -65
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = False"
    }

    redes_analisadas_4.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [-104.0, -148.0, -171.0]
Geração: 2/20, Recompensa: [-81.0, -114.0, -115.0]
Geração: 3/20, Recompensa: [-84.0, -86.0, -139.0]
Geração: 4/20, Recompensa: [-78.0, -83.0, -85.0]
Geração: 5/20, Recompensa: [-65.0, -73.0, -88.0]
Geração: 6/20, Recompensa: [-92.0, -98.0, -104.0]
Geração: 7/20, Recompensa: [-75.0, -91.0, -107.0]
Geração: 8/20, Recompensa: [-79.0, -105.0, -142.0]
Geração: 9/20, Recompensa: [-64.0, -73.0, -80.0]
Geração: 10/20, Recompensa: [-93.0, -96.0, -99.0]
Geração: 11/20, Recompensa: [-77.0, -89.0, -91.0]
Geração: 12/20, Recompensa: [-65.0, -77.0, -85.0]
Geração: 1/20, Recompensa: [-447.0, -500.0, -500.0]
Geração: 2/20, Recompensa: [-199.0, -209.0, -217.0]
Geração: 3/20, Recompensa: [-97.0, -129.0, -134.0]
Geração: 4/20, Recompensa: [-113.0, -146.0, -268.0]
Geração: 5/20, Recompensa: [-76.0, -85.0, -85.0]
Geração: 6/20, Recompensa: [-72.0, -86.0, -91.0]
Geração: 7/20, Recompensa: [-64.0, -75.0, -78.0]
Geração: 8/20, Recompensa: [-64.0, -64.0, -65

In [88]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []
    recompencas = []

    recompencas_ao_tempo = []
    Tempo_por_geracao = []

    contador_recompencas = 0
    contador_max_passos = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        inicio = time.time()

        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
    
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        verificador = sorted(recompencas, reverse=True)[:3]
        contador_max_passos += passos
        if verificador[0] >= objetivo:
            return contador_max_passos
            

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        recompencas_ao_tempo.append(recompencas)
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        fim = time.time()
        Tempo_por_geracao.append(fim - inicio)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
    
    obs = [recompencas_ao_tempo, Tempo_por_geracao]
    return selecao_final[0], obs

In [89]:
env = gym.make("CartPole-v1")

redes_analisadas_5 = []
contador_n_finalizados = 0

for c in range(100):

    tempo_inicio = time.time()
    QTDs_passos = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()
    print(QTDs_passos)
    if isinstance(QTDs_passos, int):
        redes_analisadas_5.append(QTDs_passos)
    else:
        contador_n_finalizados += 1
    
    print(f"Fim da geração {c+1}")


Geração: 1/20, Recompensa: [61.0, 35.0, 34.0]
Geração: 2/20, Recompensa: [376.0, 77.0, 67.0]
Geração: 3/20, Recompensa: [376.0, 150.0, 77.0]
Geração: 4/20, Recompensa: [376.0, 150.0, 140.0]
196
Fim da geração 1
Geração: 1/20, Recompensa: [99.0, 20.0, 14.0]
Geração: 2/20, Recompensa: [414.0, 99.0, 82.0]
Geração: 3/20, Recompensa: [421.0, 414.0, 130.0]
Geração: 4/20, Recompensa: [440.0, 421.0, 414.0]
279
Fim da geração 2
Geração: 1/20, Recompensa: [155.0, 10.0, 10.0]
Geração: 2/20, Recompensa: [155.0, 44.0, 44.0]
Geração: 3/20, Recompensa: [254.0, 175.0, 155.0]
64
Fim da geração 3
Geração: 1/20, Recompensa: [46.0, 44.0, 27.0]
Geração: 2/20, Recompensa: [100.0, 46.0, 44.0]
47
Fim da geração 4
Geração: 1/20, Recompensa: [36.0, 35.0, 27.0]
Geração: 2/20, Recompensa: [73.0, 63.0, 49.0]
Geração: 3/20, Recompensa: [73.0, 71.0, 63.0]
Geração: 4/20, Recompensa: [104.0, 96.0, 77.0]
Geração: 5/20, Recompensa: [188.0, 104.0, 96.0]
Geração: 6/20, Recompensa: [188.0, 104.0, 104.0]
Geração: 7/20, Reco

In [90]:
def calcular_media(lista):
    """Calcula a média de uma lista."""
    if not lista:
        return 0  # Retorna 0 se a lista estiver vazia para evitar divisão por zero
    return sum(lista) / len(lista)

In [93]:
print("A média da lista de passos é:", round(calcular_media(redes_analisadas_5)))
print(f"O maior numero de passos {max(redes_analisadas_5)}")
print(f"Não finalizados {contador_n_finalizados} logo o tempo é: {3500 * contador_n_finalizados}")

A média da lista de passos é: 526
O maior numero de passos 3419
Não finalizados 0 logo o tempo é: 0


In [101]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    numero_branch = n_por_geracao // 3
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []

    recompencas_ao_tempo = []
    Tempo_por_geracao = []

    contador_recompencas = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        inicio = time.time()
        recompencas = []

        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
                
        lista_geracional = random.sample(lista_geracional, numero_branch)
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        recompencas_ao_tempo.append(recompencas)
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        fim = time.time()
        Tempo_por_geracao.append(fim - inicio)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
    
    obs = [recompencas_ao_tempo, Tempo_por_geracao]
    return selecao_final[0], obs

In [102]:
env = gym.make("CartPole-v1")

redes_analisadas_6= []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = True"
    }

    redes_analisadas_4.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [116.0, 52.0, 21.0]
Geração: 2/20, Recompensa: [31.0, 19.0, 17.0]
Geração: 3/20, Recompensa: [167.0, 56.0, 48.0]
Geração: 4/20, Recompensa: [71.0, 58.0, 50.0]
Geração: 5/20, Recompensa: [95.0, 80.0, 40.0]
Geração: 6/20, Recompensa: [500.0, 244.0, 95.0]
Geração: 7/20, Recompensa: [172.0, 100.0, 98.0]
Geração: 8/20, Recompensa: [250.0, 80.0, 64.0]
Geração: 9/20, Recompensa: [141.0, 91.0, 85.0]
Geração: 10/20, Recompensa: [150.0, 95.0, 92.0]
Geração: 11/20, Recompensa: [81.0, 67.0, 61.0]
Geração: 12/20, Recompensa: [172.0, 146.0, 133.0]
Geração: 13/20, Recompensa: [125.0, 68.0, 67.0]
Geração: 14/20, Recompensa: [191.0, 111.0, 90.0]
Geração: 15/20, Recompensa: [149.0, 125.0, 119.0]
Geração: 16/20, Recompensa: [96.0, 90.0, 75.0]
Geração: 17/20, Recompensa: [128.0, 127.0, 113.0]
Geração: 18/20, Recompensa: [500.0, 240.0, 110.0]
Geração: 1/20, Recompensa: [73.0, 45.0, 14.0]
Geração: 2/20, Recompensa: [75.0, 53.0, 21.0]
Geração: 3/20, Recompensa: [159.0, 62.0, 36.0]


In [103]:
env = gym.make("CartPole-v1")

redes_analisadas_7= []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = False,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "CartPole",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = False"
    }

    redes_analisadas_4.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [47.0, 28.0, 11.0]
Geração: 2/20, Recompensa: [40.0, 27.0, 25.0]
Geração: 3/20, Recompensa: [46.0, 36.0, 12.0]
Geração: 4/20, Recompensa: [28.0, 24.0, 19.0]
Geração: 5/20, Recompensa: [25.0, 23.0, 23.0]
Geração: 6/20, Recompensa: [152.0, 121.0, 59.0]
Geração: 7/20, Recompensa: [36.0, 35.0, 31.0]
Geração: 8/20, Recompensa: [117.0, 62.0, 59.0]
Geração: 9/20, Recompensa: [103.0, 98.0, 65.0]
Geração: 10/20, Recompensa: [128.0, 73.0, 53.0]
Geração: 11/20, Recompensa: [126.0, 122.0, 103.0]
Geração: 12/20, Recompensa: [148.0, 145.0, 110.0]
Geração: 13/20, Recompensa: [101.0, 95.0, 80.0]
Geração: 14/20, Recompensa: [67.0, 56.0, 53.0]
Geração: 15/20, Recompensa: [435.0, 120.0, 99.0]
Geração: 16/20, Recompensa: [131.0, 117.0, 85.0]
Geração: 17/20, Recompensa: [110.0, 68.0, 61.0]
Geração: 18/20, Recompensa: [500.0, 93.0, 82.0]
Geração: 19/20, Recompensa: [78.0, 48.0, 46.0]
Geração: 20/20, Recompensa: [166.0, 60.0, 60.0]
Geração: 1/20, Recompensa: [23.0, 23.0, 22.0]
Gera

In [123]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    numero_branch = n_por_geracao // 3
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []

    recompencas_ao_tempo = []
    Tempo_por_geracao = []

    contador_recompencas = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        inicio = time.time()
        recompencas = []

        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
                
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)
        lista_geracional = sorted(lista_geracional, key=lambda x: x[1], reverse=True)
        lista_restante = lista_geracional[n_selecionados:]
        lista_restante = random.sample(lista_restante, numero_branch)
        lista_restante = [x for x, _ in lista_restante]

        pais_selecionados = pais_selecionados + lista_restante
        

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        recompencas_ao_tempo.append(recompencas)
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        fim = time.time()
        Tempo_por_geracao.append(fim - inicio)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
    
    obs = [recompencas_ao_tempo, Tempo_por_geracao]
    return selecao_final[0], obs

In [124]:
env = gym.make("CartPole-v1")

redes_analisadas_8 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = True"
    }

    redes_analisadas_4.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [20.0, 19.0, 15.0]
Geração: 2/20, Recompensa: [46.0, 30.0, 28.0]
Geração: 3/20, Recompensa: [96.0, 60.0, 38.0]
Geração: 4/20, Recompensa: [46.0, 41.0, 38.0]
Geração: 5/20, Recompensa: [69.0, 55.0, 46.0]
Geração: 6/20, Recompensa: [65.0, 57.0, 53.0]
Geração: 7/20, Recompensa: [161.0, 72.0, 60.0]
Geração: 8/20, Recompensa: [135.0, 86.0, 74.0]
Geração: 9/20, Recompensa: [148.0, 93.0, 93.0]
Geração: 10/20, Recompensa: [158.0, 157.0, 145.0]
Geração: 11/20, Recompensa: [267.0, 111.0, 110.0]
Geração: 12/20, Recompensa: [252.0, 95.0, 87.0]
Geração: 13/20, Recompensa: [282.0, 256.0, 126.0]
Geração: 14/20, Recompensa: [276.0, 258.0, 220.0]
Geração: 15/20, Recompensa: [277.0, 259.0, 236.0]
Geração: 16/20, Recompensa: [314.0, 234.0, 173.0]
Geração: 17/20, Recompensa: [266.0, 232.0, 220.0]
Geração: 18/20, Recompensa: [290.0, 241.0, 189.0]
Geração: 19/20, Recompensa: [279.0, 223.0, 205.0]
Geração: 20/20, Recompensa: [299.0, 265.0, 256.0]
Geração: 1/20, Recompensa: [23.0, 1

In [125]:
env = gym.make("CartPole-v1")

redes_analisadas_9= []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = False,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "CartPole",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = False"
    }

    redes_analisadas_4.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [414.0, 42.0, 18.0]
Geração: 2/20, Recompensa: [77.0, 42.0, 28.0]
Geração: 3/20, Recompensa: [193.0, 88.0, 50.0]
Geração: 4/20, Recompensa: [208.0, 101.0, 86.0]
Geração: 5/20, Recompensa: [123.0, 63.0, 56.0]
Geração: 6/20, Recompensa: [89.0, 83.0, 53.0]
Geração: 7/20, Recompensa: [305.0, 103.0, 94.0]
Geração: 8/20, Recompensa: [101.0, 76.0, 36.0]
Geração: 9/20, Recompensa: [194.0, 135.0, 103.0]
Geração: 10/20, Recompensa: [246.0, 158.0, 140.0]
Geração: 11/20, Recompensa: [269.0, 248.0, 234.0]
Geração: 12/20, Recompensa: [179.0, 121.0, 98.0]
Geração: 13/20, Recompensa: [301.0, 156.0, 124.0]
Geração: 14/20, Recompensa: [149.0, 134.0, 116.0]
Geração: 15/20, Recompensa: [160.0, 131.0, 88.0]
Geração: 16/20, Recompensa: [256.0, 185.0, 164.0]
Geração: 17/20, Recompensa: [251.0, 146.0, 126.0]
Geração: 18/20, Recompensa: [178.0, 131.0, 130.0]
Geração: 19/20, Recompensa: [214.0, 164.0, 141.0]
Geração: 20/20, Recompensa: [440.0, 115.0, 108.0]
Geração: 1/20, Recompensa: 

In [137]:
for i, lista in enumerate([redes_analisadas_1, redes_analisadas_2, redes_analisadas_3, redes_analisadas_4, redes_analisadas_5, redes_analisadas_6, redes_analisadas_7, redes_analisadas_8, redes_analisadas_9], start=1):
    print(i)
    if lista != redes_analisadas_5:
        lista = [x for _, x in lista]
        print(lista)
    nome_arquivo = f'redes_analisadas_{i}.pkl'
    salvar_dados(lista, nome_arquivo)

1
Dados salvos em redes_analisadas_1.pkl.
2
Dados salvos em redes_analisadas_2.pkl.
3
Dados salvos em redes_analisadas_3.pkl.
4
Dados salvos em redes_analisadas_4.pkl.
5
Dados salvos em redes_analisadas_5.pkl.
6
Dados salvos em redes_analisadas_6.pkl.
7
Dados salvos em redes_analisadas_7.pkl.
8
[]
Dados salvos em redes_analisadas_8.pkl.
9
[]
Dados salvos em redes_analisadas_9.pkl.


In [121]:
nome_arquivo = f'redes_analisadas_5.pkl'
redes_analisadas_1 = recarregar_dados("redes_analisadas_1.pkl")
redes_analisadas_2 = recarregar_dados("redes_analisadas_2.pkl")
redes_analisadas_3 = recarregar_dados("redes_analisadas_3.pkl")
redes_analisadas_4 = recarregar_dados("redes_analisadas_4.pkl")
redes_analisadas_5 = recarregar_dados("redes_analisadas_5.pkl")
redes_analisadas_6 = recarregar_dados("redes_analisadas_6.pkl")
redes_analisadas_7 = recarregar_dados("redes_analisadas_7.pkl")

Dados recarregados de redes_analisadas_1.pkl.
Dados recarregados de redes_analisadas_2.pkl.
Dados recarregados de redes_analisadas_3.pkl.
Dados recarregados de redes_analisadas_4.pkl.
Dados recarregados de redes_analisadas_5.pkl.
Dados recarregados de redes_analisadas_6.pkl.
Dados recarregados de redes_analisadas_7.pkl.
